In [1]:
from DataParser import DataParser
import numpy as np
import pandas as pd
import scipy.sparse as sp

from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import \
split_train_in_two_percentage_global_sample

from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
seed = 1024
parser = DataParser()

URM_all = parser.get_URM_all()
ICM_all = parser.get_ICM_all()

In [3]:
icm_frame = parser.get_icm_frame()
icm_frame

,item_id,feature_id,value
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
5,0,11171,0.074134
6,0,12276,0.183645
7,0,12899,0.110168
8,0,12996,0.205921
9,0,15509,0.131966


In [65]:
stats_frame = pd.DataFrame(data={'feature_id':icm_frame.feature_id.unique()})

In [64]:
temp=icm_frame.groupby(['feature_id']).max().rename(columns={'value': 'max_value'})['max_value'].to_frame()
temp= temp.reset_index()

In [67]:
stats_frame = stats_frame.merge(temp)

In [69]:
temp=icm_frame.groupby(['feature_id']).min().rename(columns={'value': 'min_value'})['min_value'].to_frame()
temp= temp.reset_index()
stats_frame = stats_frame.merge(temp)


,feature_id,max_value,min_value
0,1185,1.077346,0.154086
1,2507,0.984169,0.003123
2,3534,0.729615,0.202845
3,8766,0.899350,0.131871
4,10862,1.043207,0.205703
5,11171,1.050042,0.004420
6,12276,0.996266,0.056099
7,12899,0.926609,0.000654
8,12996,0.990102,0.000698
9,15509,1.006523,0.000805


In [ ]:
temp=icm_frame.groupby(['feature_id']).mean().rename(columns={'value': 'mean'})['mean'].to_frame()
temp= temp.reset_index()
stats_frame = stats_frame.merge(temp)

In [76]:
temp=icm_frame.groupby(['feature_id']).var().rename(columns={'value': 'var'})['var'].to_frame()
temp=temp.reset_index()
stats_frame = stats_frame.merge(temp)

In [77]:
stats_frame

,feature_id,max_value,min_value,mean,var
0,1185,1.077346,0.154086,0.955274,0.039872
1,2507,0.984169,0.003123,0.177363,0.019057
2,3534,0.729615,0.202845,0.342902,0.009128
3,8766,0.899350,0.131871,0.304348,0.017706
4,10862,1.043207,0.205703,0.363101,0.021003
5,11171,1.050042,0.004420,0.139913,0.014721
6,12276,0.996266,0.056099,0.269410,0.027823
7,12899,0.926609,0.000654,0.059427,0.004818
8,12996,0.990102,0.000698,0.167231,0.015245
9,15509,1.006523,0.000805,0.131975,0.009963


In [89]:
frame=icm_frame.groupby(['feature_id']).size().reset_index(name='num_assoc_item')
stats_frame = stats_frame.join(frame.set_index('feature_id'), on='feature_id')

In [90]:
stats_frame

,feature_id,max_value,min_value,mean,var,num_assoc_item
0,1185,1.077346,0.154086,0.955274,0.039872,19
1,2507,0.984169,0.003123,0.177363,0.019057,587
2,3534,0.729615,0.202845,0.342902,0.009128,39
3,8766,0.899350,0.131871,0.304348,0.017706,118
4,10862,1.043207,0.205703,0.363101,0.021003,66
5,11171,1.050042,0.004420,0.139913,0.014721,116
6,12276,0.996266,0.056099,0.269410,0.027823,217
7,12899,0.926609,0.000654,0.059427,0.004818,205
8,12996,0.990102,0.000698,0.167231,0.015245,464
9,15509,1.006523,0.000805,0.131975,0.009963,932


In [ ]:
icm_frame.groupby(['feature_id']).mean().sort_values(by='num_assoc_item', ascending=False)

In [ ]:
icm_frame=icm_frame.sort_values(by=['feature_id','num_assoc_item'])
icm_frame

In [ ]:
icm_frame=icm_frame.sort_values(by=['num_assoc_item', 'feature_id'], ascending=False)
icm_frame